# DIY Covid-19 Dashboard


In [77]:
#libraries required and imported for the following code
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [78]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

In [79]:
#allocating the file name a variable name for clarity and ease of repetitive use
filekey="admissionbeds"

In [80]:
#function to access the data from the json file
def access_json_file():
    
    with open(filekey + ".json", "rt") as INFILE:
        jsondata=json.load(INFILE)
        
    return jsondata


In [81]:

jsondata = access_json_file()


In [82]:
# converts the date string to datetime instances using pandas
def parse_date(datestring):
    return pd.to_datetime(datestring, format="%Y-%m-%d")
#wrangling api data
def wrangle_data(rawdata, type, printlogs):
    datalist=rawdata['data']
#sorting dates from rawdata taken from the api 
    dates=[dictionary["date"] for dictionary in datalist]
    dates.sort()
#checking if there are duplicate dates
    numberofuniquedates=len(set(dates))
    numberofdates=len(dates)
    aredatesunique=numberofdates==numberofuniquedates
   
#showing the date range we wanted listed in the dataframe
    startdate=parse_date(dates[0])
    enddate=parse_date(dates[-1])
    if printlogs==True:
        print("total number of dates", numberofdates, "total number of unique dates", numberofuniquedates, "on refresh this will fetch new data from the api")
        print(" are dates unique =", aredatesunique)
        print(startdate, " to ", enddate) 
#creating the dataframe, specifying, daily increment, from startdate to enddate and titling the columns (type referring to the users chosen comparison)
    index=pd.date_range(startdate, enddate, freq="D")
    admissionbedsdf=pd.DataFrame(index=index, columns=[type, "vbo"])

#adding the data to the columns of the dataframe
    for entry in datalist:
        date=parse_date(entry["date"])
        for column in [type, "vbo"]:
            if pd.isna(admissionbedsdf.loc[date, column]):
                value= float(entry[column]) if entry [column]!=None else 0.0
                admissionbedsdf.loc[date, column]=value
#fill in any entries due to missing dates 
    admissionbedsdf.fillna(0.0, inplace=True)
    return admissionbedsdf

#feeds data from the json file in to be wrangled
df=wrangle_data(jsondata, "admissions", False)

### Graph displaying admissions vs ventilator bed occupancy or cases vs ventilator bed occupancy

In [83]:
# fetching data from the api and adding it the json file
def access_api():
    
    filters = ['areaType=overview']

    structure = {
        "date" : "date",
        "admissions" : "newAdmissions",
        "vbo" : "covidOccupiedMVBeds",
        "cases" : "newCasesByPublishDate",
    }


    api = Cov19API(filters=filters, structure=structure)

    admissionbeds=api.get_json()
    return admissionbeds

In [84]:
def refresh_json_file():
    
    data=access_api()
    
    with open(filekey + ".json", "wt") as OUTF:
        json.dump(data, OUTF)
        



In [85]:

def api_button_callback(button):
#declaring jsondata as a global variable to be used else where   
    global jsondata
    jsondata=access_api()
#declaring df as a global variable to be used else where  
    global df
    
    refresh_graph()
#changing button icon once pressed  
#apibutton.icon="check"

#button appearance    
apibutton=wdg.Button(
    description='Refresh', 
    disabled=False,
    button_style='info', 
    tooltip="refresh graph",
    icon='refresh'
)

# remember to register your button callback function with the button
apibutton.on_click(api_button_callback) # the name of your function inside these brackets

display(apibutton)

# run all cells before clicking on this button

Button(button_style='info', description='Refresh', icon='refresh', style=ButtonStyle(), tooltip='refresh graph…

In [86]:
   
def plot_graph(type):
    
    df=wrangle_data(jsondata, type, True)
  
    df.plot()

#drop down menu to select comparison
typeselect=wdg.Dropdown(
    options=["admissions", "cases"],
    value="admissions",
    description= "compare",
    disabled=False
)    


#function to force the redraw of the graph with updated data
def refresh_graph():
  
    current=typeselect.value
    if current==typeselect.options[0]:
        other=typeselect.options[1]
    else:
        other=typeselect.options[0]
    typeselect.value=other 
    typeselect.value=current 
    


graph=wdg.interactive_output(plot_graph, {"type" : typeselect}) 

display(typeselect, graph)
       

Dropdown(description='compare', options=('admissions', 'cases'), value='admissions')

Output()